In [1]:
!unzip trivago-clicks.zip

Archive:  trivago-clicks.zip
   creating: trivago-clicks/
  inflating: trivago-clicks/label-names-trivago-clicks.txt  
  inflating: trivago-clicks/node-labels-trivago-clicks.txt  
  inflating: trivago-clicks/README.txt  
  inflating: trivago-clicks/hyperedges-trivago-clicks.txt  


In [7]:
!unzip cat-edge-Cooking.zip

Archive:  cat-edge-Cooking.zip
   creating: cat-edge-Cooking/
  inflating: cat-edge-Cooking/hyperedges.txt  
  inflating: cat-edge-Cooking/hyperedge-labels.txt  
  inflating: cat-edge-Cooking/hyperedge-label-identities.txt  
  inflating: cat-edge-Cooking/node-labels.txt  
  inflating: cat-edge-Cooking/README.txt  


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py



Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 592 (delta 125), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (592/592), 194.79 KiB | 6.96 MiB/s, done.
Resolving deltas: 100% (299/299), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.12 environment at: /usr
Resolved 173 packages in 3.57s
 Downloaded cuproj-cu12
 Downloaded rmm-cu12
 Downloaded shapely
 Downloaded cudf-cu12
 Downloaded pylibcugraph-cu12
 Downloaded libkvikio-cu12
 Downloaded datashader
 Downloaded dask
 Downloaded ucx-py-cu12
 Downloaded cugraph-cu12
 Downloaded bokeh
 Downloaded librmm-cu12
 Downloaded cuml-cu12
 Downloaded cuspatial-cu12
 Downloaded cucim-cu12
 Downloaded libcuspatial-cu12
 Downloaded pylibcudf-cu12
 Downloaded libraft-cu12
 Downloaded libcudf-cu12
 Downloaded raft-dask-cu12
 Downloaded libcuml-cu12
 Downloaded libcuvs

In [ ]:
import time
import cudf
import cugraph

def load_hyperedges(file_path):
    """Load hyperedges from file and convert to list of sets."""
    hypers = []
    with open(file_path, 'r') as f:
        for line in f:
            hypers.append(set(map(int, line.strip().split(','))))
    return hypers

def build_bipartite_edgelist(hyperedges):
    """
    cudf.DataFrame with columns ['src','dst'] for each (hyperedge,node) incidence.
    We number hyperedges 0..H-1 and real nodes H..H+N-1.
    """
    H = len(hyperedges)
    srcs, dsts = [], []
    for he_id, he in enumerate(hyperedges):
        for node in he:
            srcs.append(he_id)
            dsts.append(node + H)
    return cudf.DataFrame({'src': srcs, 'dst': dsts})

def run_cugraph_mst(hyperedges):
    H = len(hyperedges)

    # 1) Build bipartite graph
    edgelist = build_bipartite_edgelist(hyperedges)
    G = cugraph.Graph(directed=False)
    G.from_cudf_edgelist(edgelist, source='src', destination='dst', renumber=False)

    # 2) Compute Jaccard similarity on the bipartite graph
    jdf = cugraph.jaccard(G)

    # 3) Keep only hyperedge–hyperedge pairs (both IDs < H)
    mask = (jdf['first'] < H) & (jdf['second'] < H)
    jdf_he = jdf[mask].rename(columns={'first':'src','second':'dst'})
    # **use float64 for weight**
    jdf_he['weight'] = (1.0 - jdf_he['jaccard_coeff']).astype('float64')

    # 4) Build the sparse overlap graph (just src/dst)
    G_j = cugraph.Graph(directed=False)
    G_j.from_cudf_edgelist(jdf_he[['src','dst']],
                           source='src', destination='dst',
                           renumber=False)

    # 5) Find connected components
    comp_df = cugraph.connected_components(G_j)
    # rename whatever the second column is to 'component'
    comp_label_col = comp_df.columns[1]
    comp_df = comp_df.rename(columns={comp_label_col: 'component'})

    # 6) Identify the largest component by size
    counts_df = (comp_df
                 .groupby('component')
                 .agg(count=('vertex','count'))
                 .reset_index())
    largest_comp = int(
        counts_df.sort_values('count', ascending=False)
                 ['component']
                 .iloc[0]
    )

    # 7) Pick the highest-degree node in that largest component as the hub
    deg_df = G_j.degree().rename(columns={'vertex':'vertex','degree':'degree'})
    comp_deg = deg_df.merge(comp_df, on='vertex')
    hub_row = (comp_deg[comp_deg['component'] == largest_comp]
               .sort_values('degree', ascending=False)
               .head(1))
    hub_id = int(hub_row['vertex'].iloc[0])

    # 8) Build glue‐edges (hub → every other hyperedge outside largest comp)
    outside = comp_df[comp_df['component'] != largest_comp]['vertex']
    glue_df = cudf.DataFrame({
        'src':    cudf.Series([hub_id] * len(outside), dtype=outside.dtype),
        'dst':    outside.reset_index(drop=True),
        # **also use float64 here**
        'weight': cudf.Series([1.0] * len(outside), dtype='float64')
    })

    # 9) Combine real overlap edges + glue edges
    combined = cudf.concat(
        [jdf_he[['src','dst','weight']], glue_df],
        ignore_index=True
    )

    # 10) Build the final undirected weighted graph
    G_w = cugraph.Graph(directed=False)
    G_w.from_cudf_edgelist(combined,
                           source='src',
                           destination='dst',
                           edge_attr='weight',
                           renumber=False)

    # 11) Run Borůvka’s MST on the augmented graph
    print("Computing minimum spanning tree on GPU…")
    start = time.time()
    mst_graph = cugraph.minimum_spanning_tree(G_w, weight='weight')
    runtime = time.time() - start

    # 12) Extract edges and total weight
    mst_df = mst_graph.view_edge_list()
    edges = list(zip(
        mst_df['src'].to_pandas().tolist(),
        mst_df['dst'].to_pandas().tolist(),
        mst_df['weight'].to_pandas().tolist()
    ))
    total_weight = float(mst_df['weight'].sum())

    return {
        'edges': edges,
        'total_weight': total_weight,
        'runtime': runtime
    }

def save_tree(edges, filename):
    """Save MST edges to file."""
    with open(filename, 'w') as f:
        for u, v, w in edges:
            f.write(f"{u},{v},{w}\n")

def main():
    print("Loading hyperedges…")
    hypers = load_hyperedges('/content/trivago-clicks/hyperedges-trivago-clicks.txt')
    print(f"Loaded {len(hypers)} hyperedges")

    print("Running GPU-accelerated MST with glue edges (float64)…")
    result = run_cugraph_mst(hypers)
    print(f"Runtime        : {result['runtime']:.4f} s")
    print(f"Total weight   : {result['total_weight']:.4f}")
    print(f"Edges in MST   : {len(result['edges'])} (should be {len(hypers)-1})")

    save_tree(result['edges'], 'trivago_cugraph_mst_jaccard_glue.txt')

if __name__ == "__main__":
    main()


Loading hyperedges…
Loaded 233202 hyperedges
Running GPU-accelerated MST with glue edges (float64)…
Computing minimum spanning tree on GPU…
Runtime        : 0.4832 s
Total weight   : 144510.5540
Edges in MST   : 233201 (should be 233201)


In [ ]:
118531.0093

In [ ]:
#!/usr/bin/env python3
import math
import random
from collections import defaultdict, Counter

class DSU:
    """Disjoint-set union for finding the largest connected component."""
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n

    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        elif self.rank[ra] > self.rank[rb]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1

def load_node_labels(path):
    with open(path) as f:
        return [int(line.strip()) for line in f if line.strip()]

def load_label_names(path):
    with open(path) as f:
        return [line.rstrip("\n") for line in f]

def load_hyperedges(path):
    raw = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if line:
                raw.append([int(x) for x in line.split(",")])
    return raw

def normalize_and_keep_lcc(raw_edges, n_nodes):
    """
    1) Drop any hyperedge that references an invalid node index.
    2) Deduplicate each edge’s node list & drop singletons.
    3) Keep only the largest connected component.
    """
    # 1) filter out edges with out-of-range nodes
    filtered = []
    for e in raw_edges:
        if all(0 <= v < n_nodes for v in e):
            filtered.append(e)
    # 2) dedup and drop size<2
    clean = []
    for e in filtered:
        u = sorted(set(e))
        if len(u) >= 2:
            clean.append(u)
    # 3) find LCC via DSU
    dsu = DSU(n_nodes)
    for e in clean:
        a0 = e[0]
        for a in e[1:]:
            dsu.union(a0, a)
    # count component sizes
    comp_size = defaultdict(int)
    for i in range(n_nodes):
        comp_size[dsu.find(i)] += 1
    main = max(comp_size, key=comp_size.get)
    valid_nodes = {i for i in range(n_nodes) if dsu.find(i) == main}
    # keep only edges fully in main CC
    final = [e for e in clean if all(v in valid_nodes for v in e)]
    return final, valid_nodes

def compute_hyperedge_labels(edges, node_labels):
    """
    Assign each hyperedge the mode of its nodes’ labels.
    Tie → smallest label.
    """
    hed_labels = []
    for e in edges:
        cnt = Counter(node_labels[v] for v in e)
        mode = min((lbl for lbl, _ in cnt.items() if cnt[lbl] == max(cnt.values())))
        hed_labels.append(mode)
    return hed_labels

def stratified_sample_indices(stratum_labels, target_count, seed=42):
    """
    Stratify indices by stratum_labels[i], guarantee ≥1 per label,
    bump target_count if needed, then fill randomly.
    """
    random.seed(seed)
    m = len(stratum_labels)
    lbl2idxs = defaultdict(list)
    for i, lbl in enumerate(stratum_labels):
        lbl2idxs[lbl].append(i)
    usable = list(lbl2idxs)
    if target_count < len(usable):
        print(f"⚠️  target_count={target_count} < #labels={len(usable)}, bumping to {len(usable)}")
        target_count = len(usable)
    selected = set()
    for lbl in usable:
        selected.add(random.choice(lbl2idxs[lbl]))
        if len(selected) == target_count:
            break
    need = target_count - len(selected)
    if need > 0:
        remaining = list(set(range(m)) - selected)
        selected.update(random.sample(remaining, need))
    return selected

def write_hyperedges(eids, edges, path):
    with open(path, 'w') as f:
        for i in sorted(eids):
            f.write(",".join(map(str, edges[i])) + "\n")

def write_node_labels_from_edges(eids, edges, node_labels, path):
    nodes = set()
    for i in eids:
        nodes.update(edges[i])
    with open(path, 'w') as f:
        for v in sorted(nodes):
            f.write(f"{node_labels[v]}\n")

def write_label_names(label_names, path):
    with open(path, 'w') as f:
        for nm in label_names:
            f.write(nm + "\n")

def main():
    ### === CONFIGURE HERE ===
    node_labels_path   = '/content/trivago-clicks/node-labels-trivago-clicks.txt'
    label_names_path   = '/content/trivago-clicks/label-names-trivago-clicks.txt'
    hyperedges_path    = '/content/trivago-clicks/hyperedges-trivago-clicks.txt'
    output_prefix      = "sample10pct"
    sample_fraction    = 0.50
    random_seed        = 123
    ### ====================

    # Load
    node_labels  = load_node_labels(node_labels_path)
    label_names  = load_label_names(label_names_path)
    raw_edges    = load_hyperedges(hyperedges_path)

    # Normalize + LCC
    edges, valid_nodes = normalize_and_keep_lcc(raw_edges, len(node_labels))
    print(f"→ {len(raw_edges)} raw → {len(edges)} valid hyperedges; {len(valid_nodes)} nodes in main CC")

    # Label each hyperedge
    hed_labels = compute_hyperedge_labels(edges, node_labels)

    # Sample 10% stratified by hyperedge-label
    total = len(edges)
    target = max(1, int(math.floor(total * sample_fraction)))
    sel = stratified_sample_indices(hed_labels, target, seed=random_seed)
    print(f"→ selected {len(sel)}/{total} hyperedges ({sample_fraction*100:.1f}%)")

    # Write outputs
    write_hyperedges(sel, edges,                     f"hyperedges-{output_prefix}.txt")
    write_node_labels_from_edges(sel, edges, node_labels, f"node-labels-{output_prefix}.txt")
    write_label_names(label_names,                    f"label-names-{output_prefix}.txt")

    print("✅ Done.")
    print(f"  hyperedges-{output_prefix}.txt")
    print(f"  node-labels-{output_prefix}.txt")
    print(f"  label-names-{output_prefix}.txt")

if __name__ == "__main__":
    main()


→ 233202 raw → 233198 valid hyperedges; 172733 nodes in main CC
→ selected 23319/233198 hyperedges (10.0%)
✅ Done.
  hyperedges-sample10pct.txt
  node-labels-sample10pct.txt
  label-names-sample10pct.txt


In [1]:
import os
import time
from pathlib import Path
from typing import Dict, List, Tuple, Set

import numpy as np
import cupy as cp
import pandas as pd
from scipy.sparse import csr_matrix as sp_csr
from cupyx.scipy.sparse import csr_matrix as cp_csr
from cuml.metrics import sparse_pairwise_distances
import cugraph
from collections import Counter, defaultdict

###############################
#          SETTINGS           #
###############################
DATA_DIR = Path("/content/trivago-clicks")
DATASET_PREFIX = "trivago-clicks"  # file name stem

OUT_DIR = Path("outputs")
PARTITION_DIR = OUT_DIR / "partitions"
PARTITION_DIR.mkdir(parents=True, exist_ok=True)

###############################
#     DATA LOADING HELPERS    #
###############################

def load_hyperedges(path: Path) -> List[Set[int]]:
    """Return list of hyperedges as sets of ints (node IDs)."""
    hypers: List[Set[int]] = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if line:
                hypers.append(set(map(int, line.split(","))))
    return hypers


def load_node_labels(path: Path) -> List[str]:
    """Return list where index=node ID and value=label string."""
    with open(path) as f:
        return [ln.strip() for ln in f]


def majority_label(hyperedge: Set[int], node_labels: List[str]) -> str:
    """Label hyperedge by the most frequent node label (lexicographic tie‑break)."""
    cnt = Counter(node_labels[n] if n < len(node_labels) else "UNKNOWN" for n in hyperedge)
    if not cnt:
        return "UNKNOWN"
    max_c = max(cnt.values())
    return min(lbl for lbl, c in cnt.items() if c == max_c)

###############################
#   GRAPH + PARTITION BUILD   #
###############################

def build_incidence_and_labels() -> Tuple[sp_csr, np.ndarray, List[Set[int]]]:
    """Load dataset, fix 1‑based IDs, build CSR incidence & int labels array."""
    hyper_path = DATA_DIR / f"hyperedges-{DATASET_PREFIX}.txt"
    label_path = DATA_DIR / f"node-labels-{DATASET_PREFIX}.txt"

    hypers = load_hyperedges(hyper_path)
    node_labels = load_node_labels(label_path)

    # detect 1‑based: if ANY node id >= len(node_labels)
    n_nodes_file = len(node_labels)
    if any(n >= n_nodes_file for he in hypers for n in he):
        hypers = [set(n - 1 for n in he) for he in hypers]

    # incidence rows/cols
    rows, cols = [], []
    for hid, he in enumerate(hypers):
        rows.extend([hid] * len(he))
        cols.extend(he)
    data = np.ones(len(rows), dtype=np.int8)
    n_hyperedges = len(hypers)
    n_nodes = max(cols) + 1
    H = sp_csr((data, (rows, cols)), shape=(n_hyperedges, n_nodes))

    # string labels → ints
    str_labs = [majority_label(he, node_labels) for he in hypers]
    uniq = {lbl: i for i, lbl in enumerate(sorted(set(str_labs)))}
    int_labels = np.array([uniq[lbl] for lbl in str_labs], dtype=np.int32)
    return H, int_labels, hypers

###############################
#   INTERNAL MST (GPU Prim)   #
###############################

def prim_exact_mst(X_csr: cp_csr, metric: str = "jaccard") -> List[Tuple[int, int, float]]:
    n = X_csr.shape[0]
    if n == 1:
        return []
    in_tree = cp.zeros(n, dtype=cp.bool_)
    in_tree[0] = True

    dist = sparse_pairwise_distances(X_csr[0:1], X_csr, metric=metric).ravel()
    parent = cp.zeros(n, dtype=cp.int32)
    edges = []
    for _ in range(1, n):
        u = int(cp.argmin(cp.where(in_tree, cp.inf, dist)).get())
        edges.append((int(parent[u].get()), u, float(dist[u].get())))
        in_tree[u] = True
        du = sparse_pairwise_distances(X_csr[u:u+1], X_csr, metric=metric).ravel()
        mask = (~in_tree) & (du < dist)
        dist = cp.where(mask, du, dist)
        parent = cp.where(mask, u, parent)
    return edges

###############################
#   COARSENED GRAPH HELPERS   #
###############################

def min_distance(hypers: List[Set[int]], jdict: Dict[Tuple[int, int], float], A: List[int], rep_B: int) -> float:
    """Min distance from any hyperedge in A to representative rep_B."""
    return min(jdict.get((u, rep_B), jdict.get((rep_B, u), 1.0)) for u in A)

###############################
#              MAIN           #
###############################

def main():
    t0 = time.perf_counter()
    H, labels, hypers = build_incidence_and_labels()
    load_time = time.perf_counter() - t0

    # build partitions
    part: Dict[int, List[int]] = defaultdict(list)
    for idx, lab in enumerate(labels):
        part[int(lab)].append(idx)

    # compute Jaccard matrix for entire set using helper from original pipeline
    from typing import Dict as _Dict, Tuple as _Tuple
    def build_bipartite_df(hs: List[Set[int]]):
        src, dst = [], []
        HN = len(hs)
        for hid, he in enumerate(hs):
            for n in he:
                src.append(hid)
                dst.append(n + HN)
        return cudf.DataFrame({"src": src, "dst": dst})

    def get_global_jaccard(hs: List[Set[int]]) -> _Dict[_Tuple[int,int], float]:
        HN = len(hs)
        if HN <= 1:
            return {}
        df = build_bipartite_df(hs)
        G = cugraph.Graph(directed=False)
        G.from_cudf_edgelist(df, source='src', destination='dst', renumber=False)
        jdf = cugraph.jaccard(G)
        mask = (jdf['first'] < HN) & (jdf['second'] < HN)
        jdf = jdf[mask]
        jdf['weight'] = (1.0 - jdf['jaccard_coeff']).astype('float32')
        pdf = jdf[['first','second','weight']].to_pandas()
        return {(int(u), int(v)): float(w) for u, v, w in pdf.itertuples(index=False)}

    jdict = get_global_jaccard(hypers)

    # Component MSTs
    H_gpu = cp_csr(H.astype(np.float32))
    comp_edges: List[Tuple[int,int,float]] = []
    comp_weight = 0.0
    for lab, idxs in part.items():
        sub_gpu = H_gpu[idxs]
        e = prim_exact_mst(sub_gpu)
        comp_edges.extend([(idxs[s], idxs[d], w) for s,d,w in e])
        comp_weight += sum(w for _,_,w in e)

    # Coarsened MST
    reps = {lab: idxs[0] for lab, idxs in part.items()}
    labs = list(part.keys())
    rows = []; cols = []; wts = []
    for i, li in enumerate(labs[:-1]):
        Pi = part[li]
        for j, lj in enumerate(labs[i+1:], i+1):
            Pj = part[lj]
            wi = min_distance(hypers, jdict, Pi, reps[lj])
            wj = min_distance(hypers, jdict, Pj, reps[li])
            w = min(wi, wj, 1.0)
            rows.append(i); cols.append(j); wts.append(w)
    from scipy.sparse.csgraph import minimum_spanning_tree
    M = sp_csr((wts, (rows, cols)), shape=(len(labs), len(labs)))
    M = M + M.T
    mst_small = minimum_spanning_tree(M).toarray()
    coarse_edges = []
    coarse_weight = 0.0
    for i in range(len(labs)):
        for j in range(len(labs)):
            w = mst_small[i, j]
            if w > 0:
                coarse_edges.append((reps[labs[i]], reps[labs[j]], float(w)))
                coarse_weight += w

    # Combined
    total_mfc = comp_weight + coarse_weight

    print(f"Load time: {load_time:.2f}s  |  Hyperedges: {len(hypers):,}  Partitions: {len(part)}")
    print(f"Component MST weight: {comp_weight:.4f}  |  Coarse MST weight: {coarse_weight:.4f}  |  MFC total: {total_mfc:.4f}")


if __name__ == "__main__":
    main()


Loading data...
233,202 hyperedges, 144 partitions
1) Computing exact global MST...
   OPT MST weight = 144510.5540
2) Computing partition MSTs...
   Sum cluster MST weights = 166507.0513
3) Computing coarsened MST...
   Coarse MST weight = 90.3475
4) MFC total weight = 166597.39885097742
   beta (MFC/OPT)    = 1.1528389746457437
   gamma (intra)     = 3.1548123102593326


In [1]:
import cupy as cp
from cuml.metrics import sparse_pairwise_distances
import time
from tqdm import tqdm
import numpy as np
import cupy as cp
from scipy.sparse import csr_matrix as sp_csr
from cupyx.scipy.sparse import csr_matrix as cp_csr
from cuml.metrics import sparse_pairwise_distances
from cuml.neighbors import NearestNeighbors
import cudf
import cugraph

# 1) Load your hyperedges.txt into a SciPy CSR matrix
#    – each line of hyperedges.txt is a whitespace-separated list of ingredient IDs
rows = []
cols = []
with open('/content/cat-edge-Cooking/hyperedges.txt', 'r') as f:
    for i, line in enumerate(f):
        ing = [int(x) for x in line.strip().split()]
        rows.extend([i] * len(ing))
        cols.extend(ing)
data = np.ones(len(rows), dtype=np.int8)
n_recipes = i + 1
n_ingredients = max(cols) + 1
H = sp_csr((data, (rows, cols)),
           shape=(n_recipes, n_ingredients))
def prim_exact_mst(X_csr, metric='jaccard'):
    n = X_csr.shape[0]
    in_tree = cp.zeros(n, dtype=cp.bool_)
    in_tree[0] = True

    # dist[v] = best distance from v to any node in the current tree
    dist   = cp.full(n, cp.inf, dtype=cp.float32)
    parent = cp.full(n, -1, dtype=cp.int32)

    # initialize distances from node 0
    d0 = sparse_pairwise_distances(X_csr[0:1], X_csr, metric=metric).ravel()
    dist = d0
    parent[:] = 0

    edges = []
    for _ in tqdm(range(1, n)):
        # pick the closest outside-tree node
        dists = cp.where(in_tree, cp.inf, dist)
        u = int(cp.argmin(dists).get())

        # record the MST edge (parent[u] → u)
        edges.append((int(parent[u].get()), u, float(dist[u].get())))
        in_tree[u] = True

        # update distances using the newly added node u
        du = sparse_pairwise_distances(X_csr[u:u+1], X_csr, metric=metric).ravel()
        mask = (~in_tree) & (du < dist)
        dist   = cp.where(mask, du, dist)
        parent = cp.where(mask, u, parent)

    return edges  # list of (src, dst, weight)

# Usage:
from cupyx.scipy.sparse import csr_matrix as cp_csr
H_gpu = cp_csr(H.astype(float))
mst_edges = prim_exact_mst(H_gpu, metric='jaccard')


/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:213: UserWarning: Y was converted to boolean for metric jaccard
  ret = func(*args, **kwargs)
  0%|          | 11/39773 [00:00<12:04, 54.87it/s]/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:213: UserWarning: X was converted to boolean for metric jaccard
  ret = func(*args, **kwargs)
100%|██████████| 39773/39773 [05:14<00:00, 126.39it/s]


In [2]:
# save prims tree
w = 0
with open('prims_tree_cooking.txt', 'w') as f:

    for edge in mst_edges:
        w += edge[2]
        f.write(f"{edge[0]} {edge[1]} {edge[2]}\n")

In [3]:
print(w)

24054.699267568587


In [11]:
print(w)

2793.5656913811167


In [6]:
import os
import time
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import cupy as cp
import pandas as pd
from scipy.sparse import csr_matrix as sp_csr
from cupyx.scipy.sparse import csr_matrix as cp_csr
from cuml.metrics import sparse_pairwise_distances
import cugraph

###############################
#          SETTINGS           #
###############################
DATA_DIR = Path("/content/cat-edge-Cooking")  # change if needed
OUT_DIR = Path("outputs")
PARTITION_DIR = OUT_DIR / "partitions"
PARTITION_DIR.mkdir(parents=True, exist_ok=True)

###############################
#     UTILITY  FUNCTIONS     #
###############################

def load_hypergraph() -> Tuple[sp_csr, np.ndarray]:
    """Load hyperedges CSR matrix H and hyperedge labels."""
    rows, cols = [], []
    with open(DATA_DIR / "hyperedges.txt", "r") as f:
        for i, line in enumerate(f):
            ing = [int(x) for x in line.strip().split()]
            rows.extend([i] * len(ing))
            cols.extend(ing)
    data = np.ones(len(rows), dtype=np.int8)
    n_hyperedges = i + 1
    n_nodes = max(cols) + 1
    H = sp_csr((data, (rows, cols)), shape=(n_hyperedges, n_nodes))

    labels = np.loadtxt(DATA_DIR / "hyperedge-labels.txt", dtype=np.int32)
    assert labels.shape[0] == n_hyperedges, "Label / hyperedge size mismatch!"
    return H, labels


def prim_exact_mst(X_csr: cp_csr, metric: str = "jaccard") -> List[Tuple[int, int, float]]:
    """GPU‑accelerated Prim's algorithm; returns edges as (src, dst, weight)."""
    n = X_csr.shape[0]
    in_tree = cp.zeros(n, dtype=cp.bool_)
    in_tree[0] = True

    dist = sparse_pairwise_distances(X_csr[0:1], X_csr, metric=metric).ravel()
    parent = cp.zeros(n, dtype=cp.int32)

    edges: List[Tuple[int, int, float]] = []
    for _ in range(1, n):
        u = int(cp.argmin(cp.where(in_tree, cp.inf, dist)).get())
        edges.append((int(parent[u].get()), u, float(dist[u].get())))
        in_tree[u] = True
        du = sparse_pairwise_distances(X_csr[u:u + 1], X_csr, metric=metric).ravel()
        mask = (~in_tree) & (du < dist)
        dist = cp.where(mask, du, dist)
        parent = cp.where(mask, u, parent)
    return edges


def build_partitions(labels: np.ndarray) -> Dict[int, List[int]]:
    partitions: Dict[int, List[int]] = {}
    for idx, lab in enumerate(labels):
        partitions.setdefault(lab, []).append(idx)
    return partitions


def component_representatives(partitions: Dict[int, List[int]]) -> Dict[int, int]:
    return {lab: nodes[0] for lab, nodes in partitions.items()}


def min_distance_to_rep(X_gpu: cp_csr, nodes: List[int], rep_idx: int) -> float:
    sub = X_gpu[nodes]
    rep = X_gpu[rep_idx:rep_idx + 1]
    d = sparse_pairwise_distances(rep, sub, metric="jaccard")
    return float(d.min().get())


def coarsen_graph(H_gpu: cp_csr, partitions: Dict[int, List[int]]) -> List[Tuple[int, int, float]]:
    reps = component_representatives(partitions)
    labels = sorted(partitions.keys())
    edges = []
    for i_idx, li in enumerate(labels[:-1]):
        for lj in labels[i_idx + 1:]:
            wi_j = min_distance_to_rep(H_gpu, partitions[li], reps[lj])
            wj_i = min_distance_to_rep(H_gpu, partitions[lj], reps[li])
            edges.append((li, lj, min(wi_j, wj_i)))
    return edges


def mst_on_coarsened(num_components: int, edges: List[Tuple[int, int, float]]) -> List[Tuple[int, int, float]]:
    """Run MST on coarsened graph using cuGraph and return edge list tuples."""
    if num_components <= 1:
        return []

    import cudf
    gdf = cudf.DataFrame(edges, columns=["src", "dst", "weight"])
    G = cugraph.Graph()
    G.from_cudf_edgelist(gdf, source="src", destination="dst", edge_attr="weight", renumber=False)

    # cuGraph returns a Graph; extract its edge list
    mst_graph = cugraph.minimum_spanning_tree(G)
    elist = mst_graph.view_edge_list()

    # ensure consistent column names
    elist = elist.rename(columns={"weights": "weight"})

    return list(elist.to_pandas()[["src", "dst", "weight"]].itertuples(index=False, name=None))

###############################
#           MAIN              #
###############################

def main():
    timings = {}

    # STEP 0: load data
    t0 = time.perf_counter()
    H, labels = load_hypergraph()
    timings["load_data"] = time.perf_counter() - t0

    H_gpu = cp_csr(H.astype(np.float32))

    # STEP 1: partition hyperedges
    t0 = time.perf_counter()
    partitions = build_partitions(labels)
    timings["partition"] = time.perf_counter() - t0

    mapping_path = OUT_DIR / "hyperedge_partition_mapping.txt"
    with open(mapping_path, "w") as f:
        for h_idx, lab in enumerate(labels):
            f.write(f"{h_idx}\t{lab}\n")

    # STEP 2: internal MSTs
    internal_edges: Dict[int, List[Tuple[int, int, float]]] = {}
    internal_weights: Dict[int, float] = {}
    mst_time = 0.0
    for lab, nodes in partitions.items():
        if len(nodes) == 1:
            internal_edges[lab] = []
            internal_weights[lab] = 0.0
            continue
        t0 = time.perf_counter()
        sub_gpu = H_gpu[nodes]
        edges = prim_exact_mst(sub_gpu)
        mst_time += time.perf_counter() - t0
        internal_edges[lab] = [(nodes[s], nodes[d], w) for s, d, w in edges]
        internal_weights[lab] = sum(w for _, _, w in edges)
        (PARTITION_DIR / f"partition_{lab}_mst.txt").write_text(
            "\n".join(f"{s} {d} {w}" for s, d, w in internal_edges[lab]))

    timings["component_msts"] = mst_time

    # STEP 3: coarsen + MST
    t0 = time.perf_counter()
    coarsened_edges = coarsen_graph(H_gpu, partitions)
    timings["compute_coarsened_edges"] = time.perf_counter() - t0

    t0 = time.perf_counter()
    comp_mst_edges = mst_on_coarsened(len(partitions), coarsened_edges)
    timings["mst_coarsened"] = time.perf_counter() - t0

    # STEP 4: combine
    combined_edges = [edge for edges in internal_edges.values() for edge in edges]
    combined_edges.extend(comp_mst_edges)

    approx_mfc_weight = sum(w for *_, w in combined_edges)
    coarsened_weight = sum(internal_weights.values())

    (OUT_DIR / "combined_mst.txt").write_text(
        "\n".join(f"{s} {d} {w}" for s, d, w in combined_edges))

    stats_lines = [
        *[f"{k}: {v:.4f} s" for k, v in timings.items()],
        f"approx_MFC_weight: {approx_mfc_weight:.6f}",
        f"coarsened_graph_weight: {coarsened_weight:.6f}",
    ]
    (OUT_DIR / "run_stats.txt").write_text("\n".join(stats_lines))

    print("=== Timings (s) ===")
    for k, v in timings.items():
        print(f"{k:<25} {v:.4f}")
    print("\napprox MFC weight:", approx_mfc_weight)
    print("coarsened graph weight:", coarsened_weight)


if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:213: UserWarning: Y was converted to boolean for metric jaccard
  ret = func(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:213: UserWarning: X was converted to boolean for metric jaccard
  ret = func(*args, **kwargs)


=== Timings (s) ===
load_data                 0.3632
partition                 0.0129
component_msts            149.9735
compute_coarsened_edges   1.3936
mst_coarsened             0.0984

approx MFC weight: 25084.849987387657
coarsened graph weight: 25074.57403945923


In [7]:
!zip -r /content/outputs_cooking.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/partitions/ (stored 0%)
  adding: content/outputs/partitions/partition_19_mst.txt (deflated 72%)
  adding: content/outputs/partitions/partition_8_mst.txt (deflated 73%)
  adding: content/outputs/partitions/partition_4_mst.txt (deflated 73%)
  adding: content/outputs/partitions/partition_10_mst.txt (deflated 73%)
  adding: content/outputs/partitions/partition_16_mst.txt (deflated 74%)
  adding: content/outputs/partitions/partition_17_mst.txt (deflated 74%)
  adding: content/outputs/partitions/partition_5_mst.txt (deflated 72%)
  adding: content/outputs/partitions/partition_1_mst.txt (deflated 73%)
  adding: content/outputs/partitions/partition_2_mst.txt (deflated 72%)
  adding: content/outputs/partitions/partition_20_mst.txt (deflated 75%)
  adding: content/outputs/partitions/partition_18_mst.txt (deflated 74%)
  adding: content/outputs/partitions/partition_15_mst.txt (deflated 74%)
  adding: content/outputs/partitions/par

In [17]:
import os
import time
from pathlib import Path
from typing import Dict, List, Tuple, Set

import numpy as np
import cupy as cp
import pandas as pd
from scipy.sparse import csr_matrix as sp_csr
from cupyx.scipy.sparse import csr_matrix as cp_csr
from cuml.metrics import sparse_pairwise_distances
import cugraph
from collections import Counter, defaultdict

###############################
#          SETTINGS           #
###############################
DATA_DIR = Path("/content/trivago-clicks")
DATASET_PREFIX = "trivago-clicks"  # file name stem

OUT_DIR = Path("outputs_trivago")
PARTITION_DIR = OUT_DIR / "partitions"
PARTITION_DIR.mkdir(parents=True, exist_ok=True)

###############################
#     DATA LOADING HELPERS    #
###############################

def load_hyperedges(path: Path) -> List[Set[int]]:
    """Return list of hyperedges as sets of ints (node IDs)."""
    hypers: List[Set[int]] = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if line:
                hypers.append(set(map(int, line.split(","))))
    return hypers


def load_node_labels(path: Path) -> List[str]:
    """Return list where index=node ID and value=label string."""
    with open(path) as f:
        return [ln.strip() for ln in f]


def majority_label(hyperedge: Set[int], node_labels: List[str]) -> str:
    """Label hyperedge by the most frequent node label (lexicographic tie‑break)."""
    cnt = Counter(node_labels[n] if n < len(node_labels) else "UNKNOWN" for n in hyperedge)
    if not cnt:
        return "UNKNOWN"
    max_c = max(cnt.values())
    return min(lbl for lbl, c in cnt.items() if c == max_c)

###############################
#   GRAPH + PARTITION BUILD   #
###############################

def build_incidence_and_labels() -> Tuple[sp_csr, np.ndarray, List[Set[int]]]:
    """Load dataset, fix 1‑based IDs, build CSR incidence & int labels array."""
    hyper_path = DATA_DIR / f"hyperedges-{DATASET_PREFIX}.txt"
    label_path = DATA_DIR / f"node-labels-{DATASET_PREFIX}.txt"

    hypers = load_hyperedges(hyper_path)
    node_labels = load_node_labels(label_path)

    # detect 1‑based: if ANY node id >= len(node_labels)
    n_nodes_file = len(node_labels)
    if any(n >= n_nodes_file for he in hypers for n in he):
        hypers = [set(n - 1 for n in he) for he in hypers]

    # incidence rows/cols
    rows, cols = [], []
    for hid, he in enumerate(hypers):
        rows.extend([hid] * len(he))
        cols.extend(he)
    data = np.ones(len(rows), dtype=np.int8)
    n_hyperedges = len(hypers)
    n_nodes = max(cols) + 1
    H = sp_csr((data, (rows, cols)), shape=(n_hyperedges, n_nodes))

    # string labels → ints
    str_labs = [majority_label(he, node_labels) for he in hypers]
    uniq = {lbl: i for i, lbl in enumerate(sorted(set(str_labs)))}
    int_labels = np.array([uniq[lbl] for lbl in str_labs], dtype=np.int32)
    return H, int_labels, hypers

###############################
#   INTERNAL MST (GPU Prim)   #
###############################

def prim_exact_mst(X_csr: cp_csr, metric: str = "jaccard") -> List[Tuple[int, int, float]]:
    n = X_csr.shape[0]
    if n == 1:
        return []
    in_tree = cp.zeros(n, dtype=cp.bool_)
    in_tree[0] = True

    dist = sparse_pairwise_distances(X_csr[0:1], X_csr, metric=metric).ravel()
    parent = cp.zeros(n, dtype=cp.int32)
    edges = []
    for _ in range(1, n):
        u = int(cp.argmin(cp.where(in_tree, cp.inf, dist)).get())
        edges.append((int(parent[u].get()), u, float(dist[u].get())))
        in_tree[u] = True
        du = sparse_pairwise_distances(X_csr[u:u+1], X_csr, metric=metric).ravel()
        mask = (~in_tree) & (du < dist)
        dist = cp.where(mask, du, dist)
        parent = cp.where(mask, u, parent)
    return edges

###############################
#   COARSENED GRAPH HELPERS   #
###############################

def min_distance(hypers: List[Set[int]], jdict: Dict[Tuple[int, int], float], A: List[int], rep_B: int) -> float:
    """Min distance from any hyperedge in A to representative rep_B."""
    return min(jdict.get((u, rep_B), jdict.get((rep_B, u), 1.0)) for u in A)

###############################
#              MAIN           #
###############################

def main():
    t0 = time.perf_counter()
    H, labels, hypers = build_incidence_and_labels()
    load_time = time.perf_counter() - t0

    # build partitions
    part: Dict[int, List[int]] = defaultdict(list)
    for idx, lab in enumerate(labels):
        part[int(lab)].append(idx)

    # compute Jaccard matrix for entire set using helper from original pipeline
    from typing import Dict as _Dict, Tuple as _Tuple
    def build_bipartite_df(hs: List[Set[int]]):
        src, dst = [], []
        HN = len(hs)
        for hid, he in enumerate(hs):
            for n in he:
                src.append(hid)
                dst.append(n + HN)
        return cudf.DataFrame({"src": src, "dst": dst})

    def get_global_jaccard(hs: List[Set[int]]) -> _Dict[_Tuple[int,int], float]:
        HN = len(hs)
        if HN <= 1:
            return {}
        df = build_bipartite_df(hs)
        G = cugraph.Graph(directed=False)
        G.from_cudf_edgelist(df, source='src', destination='dst', renumber=False)
        jdf = cugraph.jaccard(G)
        mask = (jdf['first'] < HN) & (jdf['second'] < HN)
        jdf = jdf[mask]
        jdf['weight'] = (1.0 - jdf['jaccard_coeff']).astype('float32')
        pdf = jdf[['first','second','weight']].to_pandas()
        return {(int(u), int(v)): float(w) for u, v, w in pdf.itertuples(index=False)}

    jdict = get_global_jaccard(hypers)

    # Component MSTs
    H_gpu = cp_csr(H.astype(np.float32))
    comp_edges: List[Tuple[int,int,float]] = []
    comp_weight = 0.0
    for lab, idxs in part.items():
        sub_gpu = H_gpu[idxs]
        e = prim_exact_mst(sub_gpu)
        comp_edges.extend([(idxs[s], idxs[d], w) for s,d,w in e])
        comp_weight += sum(w for _,_,w in e)

    # Coarsened MST
    reps = {lab: idxs[0] for lab, idxs in part.items()}
    labs = list(part.keys())
    rows = []; cols = []; wts = []
    for i, li in enumerate(labs[:-1]):
        Pi = part[li]
        for j, lj in enumerate(labs[i+1:], i+1):
            Pj = part[lj]
            wi = min_distance(hypers, jdict, Pi, reps[lj])
            wj = min_distance(hypers, jdict, Pj, reps[li])
            w = min(wi, wj, 1.0)
            rows.append(i); cols.append(j); wts.append(w)
    from scipy.sparse.csgraph import minimum_spanning_tree
    M = sp_csr((wts, (rows, cols)), shape=(len(labs), len(labs)))
    M = M + M.T
    mst_small = minimum_spanning_tree(M).toarray()
    coarse_edges = []
    coarse_weight = 0.0
    for i in range(len(labs)):
        for j in range(len(labs)):
            w = mst_small[i, j]
            if w > 0:
                coarse_edges.append((reps[labs[i]], reps[labs[j]], float(w)))
                coarse_weight += w

    # Combined
    total_mfc = comp_weight + coarse_weight

    print(f"Load time: {load_time:.2f}s  |  Hyperedges: {len(hypers):,}  Partitions: {len(part)}")
    print(f"Component MST weight: {comp_weight:.4f}  |  Coarse MST weight: {coarse_weight:.4f}  |  MFC total: {total_mfc:.4f}")


if __name__ == "__main__":
    main()


Load time: 3.48s  |  Hyperedges: 233,202  Partitions: 150
Component MST weight: 144723.8125  |  Coarse MST weight: 132.5172  |  MFC total: 144856.3297


In [18]:
!zip '/content/outputs_trivago.zip' '/content/outputs_trivago'

  adding: content/outputs_trivago/ (stored 0%)


In [24]:
import os
import time
from pathlib import Path
from typing import Dict, List, Tuple, Set

import numpy as np
import cupy as cp
import pandas as pd
from scipy.sparse import csr_matrix as sp_csr
from cupyx.scipy.sparse import csr_matrix as cp_csr
from cuml.metrics import sparse_pairwise_distances
import cugraph
from collections import Counter, defaultdict

###############################
#          SETTINGS           #
###############################
DATA_DIR = Path("/content/trivago-clicks")
DATASET_PREFIX = "trivago-clicks"  # file name stem

OUT_DIR = Path("outputs_trivago")
PARTITION_DIR = OUT_DIR / "partitions"
PARTITION_DIR.mkdir(parents=True, exist_ok=True)

###############################
#     DATA LOADING HELPERS    #
###############################

def load_hyperedges(path: Path) -> List[Set[int]]:
    """Return list of hyperedges as sets of ints (node IDs)."""
    hypers: List[Set[int]] = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if line:
                hypers.append(set(map(int, line.split(","))))
    return hypers


def load_node_labels(path: Path) -> List[str]:
    """Return list where index=node ID and value=label string."""
    with open(path) as f:
        return [ln.strip() for ln in f]


def majority_label(hyperedge: Set[int], node_labels: List[str]) -> str:
    """Label hyperedge by the most frequent node label (lexicographic tie‑break)."""
    cnt = Counter(node_labels[n] if n < len(node_labels) else "UNKNOWN" for n in hyperedge)
    if not cnt:
        return "UNKNOWN"
    max_c = max(cnt.values())
    return min(lbl for lbl, c in cnt.items() if c == max_c)

###############################
#   GRAPH + PARTITION BUILD   #
###############################

def build_incidence_and_labels() -> Tuple[sp_csr, np.ndarray, List[Set[int]]]:
    """Load dataset, fix 1‑based IDs, build CSR incidence & int labels array."""
    hyper_path = DATA_DIR / f"hyperedges-{DATASET_PREFIX}.txt"
    label_path = DATA_DIR / f"node-labels-{DATASET_PREFIX}.txt"

    hypers = load_hyperedges(hyper_path)
    node_labels = load_node_labels(label_path)

    # detect 1‑based: if ANY node id >= len(node_labels)
    n_nodes_file = len(node_labels)
    if any(n >= n_nodes_file for he in hypers for n in he):
        hypers = [set(n - 1 for n in he) for he in hypers]

    # incidence rows/cols
    rows, cols = [], []
    for hid, he in enumerate(hypers):
        rows.extend([hid] * len(he))
        cols.extend(he)
    data = np.ones(len(rows), dtype=np.int8)
    n_hyperedges = len(hypers)
    n_nodes = max(cols) + 1
    H = sp_csr((data, (rows, cols)), shape=(n_hyperedges, n_nodes))

    # string labels → ints
    str_labs = [majority_label(he, node_labels) for he in hypers]
    uniq = {lbl: i for i, lbl in enumerate(sorted(set(str_labs)))}
    int_labels = np.array([uniq[lbl] for lbl in str_labs], dtype=np.int32)
    return H, int_labels, hypers

###############################
#   INTERNAL MST (GPU Prim)   #
###############################

def prim_exact_mst(X_csr: cp_csr, metric: str = "jaccard") -> List[Tuple[int, int, float]]:
    n = X_csr.shape[0]
    if n == 1:
        return []
    in_tree = cp.zeros(n, dtype=cp.bool_)
    in_tree[0] = True

    dist = sparse_pairwise_distances(X_csr[0:1], X_csr, metric=metric).ravel()
    parent = cp.zeros(n, dtype=cp.int32)
    edges = []
    for _ in range(1, n):
        u = int(cp.argmin(cp.where(in_tree, cp.inf, dist)).get())
        edges.append((int(parent[u].get()), u, float(dist[u].get())))
        in_tree[u] = True
        du = sparse_pairwise_distances(X_csr[u:u+1], X_csr, metric=metric).ravel()
        mask = (~in_tree) & (du < dist)
        dist = cp.where(mask, du, dist)
        parent = cp.where(mask, u, parent)
    return edges

###############################
#   COARSENED GRAPH HELPERS   #
###############################

def min_distance(hypers: List[Set[int]], jdict: Dict[Tuple[int, int], float], A: List[int], rep_B: int) -> float:
    """Min distance from any hyperedge in A to representative rep_B."""
    return min(jdict.get((u, rep_B), jdict.get((rep_B, u), 1.0)) for u in A)

###############################
#              MAIN           #
###############################

def main():
    timings: Dict[str, float] = {}

    # STEP 0: load + preprocess -------------------------------------------------
    t0 = time.perf_counter()
    H, labels, hypers = build_incidence_and_labels()
    timings["load_and_preprocess"] = time.perf_counter() - t0

    # STEP 1: build partitions --------------------------------------------------
    t0 = time.perf_counter()
    partitions: Dict[int, List[int]] = defaultdict(list)
    for idx, lab in enumerate(labels):
        partitions[int(lab)].append(idx)
    timings["partition"] = time.perf_counter() - t0

    # persist partition mapping
    OUT_DIR.mkdir(exist_ok=True)
    mapping_path = OUT_DIR / "hyperedge_partition_mapping.txt"
    with open(mapping_path, "w") as f:
        for h_idx, lab in enumerate(labels):
            f.write(f"{h_idx}	{lab}")

    # STEP 2: internal MSTs -----------------------------------------------------
    H_gpu = cp_csr(H.astype(np.float32))
    internal_edges: Dict[int, List[Tuple[int, int, float]]] = {}
    internal_weights: Dict[int, float] = {}
    mst_time = 0.0
    for lab, nodes in partitions.items():
        if len(nodes) == 1:
            internal_edges[lab] = []
            internal_weights[lab] = 0.0
            continue
        t0 = time.perf_counter()
        sub_gpu = H_gpu[nodes]
        edges = prim_exact_mst(sub_gpu)
        mst_time += time.perf_counter() - t0
        internal_edges[lab] = [(nodes[s], nodes[d], w) for s, d, w in edges]
        internal_weights[lab] = sum(w for _, _, w in edges)
        # write each partition MST
        (PARTITION_DIR / f"partition_{lab}_mst.txt").write_text(
            "".join(f"{s} {d} {w}" for s, d, w in internal_edges[lab]))
    timings["component_msts"] = mst_time

    # STEP 3: coarsen graph + MST ----------------------------------------------
    # build global jaccard dict (needed for coarsening distances)
    from typing import Dict as _Dict, Tuple as _Tuple
    def build_bipartite_df(hs: List[Set[int]]):
        src, dst = [], []
        HN = len(hs)
        for hid, he in enumerate(hs):
            for n in he:
                src.append(hid)
                dst.append(n + HN)
        return cudf.DataFrame({"src": src, "dst": dst})

    def get_global_jaccard(hs: List[Set[int]]) -> _Dict[_Tuple[int,int], float]:
        HN = len(hs)
        if HN <= 1:
            return {}
        df = build_bipartite_df(hs)
        G = cugraph.Graph(directed=False)
        G.from_cudf_edgelist(df, source='src', destination='dst', renumber=False)
        jdf = cugraph.jaccard(G)
        mask = (jdf['first'] < HN) & (jdf['second'] < HN)
        jdf = jdf[mask]
        jdf['weight'] = (1.0 - jdf['jaccard_coeff']).astype('float32')
        pdf = jdf[['first','second','weight']].to_pandas()
        return {(int(u), int(v)): float(w) for u, v, w in pdf.itertuples(index=False)}

    t0 = time.perf_counter()
    jdict = get_global_jaccard(hypers)

    # reps
    reps = {lab: nodes[0] for lab, nodes in partitions.items()}
    labs = list(partitions.keys())

    rows = []; cols = []; wts = []
    for i, li in enumerate(labs[:-1]):
        Pi = partitions[li]
        for j, lj in enumerate(labs[i+1:], i+1):
            Pj = partitions[lj]
            wi = min_distance(hypers, jdict, Pi, reps[lj])
            wj = min_distance(hypers, jdict, Pj, reps[li])
            w = min(wi, wj, 1.0)
            rows.append(i); cols.append(j); wts.append(w)
    from scipy.sparse.csgraph import minimum_spanning_tree
    M = sp_csr((wts, (rows, cols)), shape=(len(labs), len(labs)))
    M = M + M.T
    timings["compute_coarsened_edges"] = time.perf_counter() - t0

    t0 = time.perf_counter()
    mst_small = minimum_spanning_tree(M).toarray()
    comp_mst_edges: List[Tuple[int,int,float]] = []
    for i in range(len(labs)):
        for j in range(len(labs)):
            w = mst_small[i, j]
            if w > 0:
                comp_mst_edges.append((reps[labs[i]], reps[labs[j]], float(w)))
    timings["mst_coarsened"] = time.perf_counter() - t0

    # STEP 4: combine ----------------------------------------------------------
    combined_edges = [e for edges in internal_edges.values() for e in edges]
    combined_edges.extend(comp_mst_edges)

    approx_mfc_weight = sum(w for *_, w in combined_edges)
    coarsened_weight = sum(internal_weights.values())

    (OUT_DIR / "combined_mst.txt").write_text(
        "".join(f"{s} {d} {w}" for s, d, w in combined_edges))

    stats_lines = [
        *[f"{k}: {v:.4f} s" for k, v in timings.items()],
        f"approx_MFC_weight: {approx_mfc_weight:.6f}",
        f"coarsened_graph_weight: {coarsened_weight:.6f}",
    ]
    (OUT_DIR / "run_stats.txt").write_text("".join(stats_lines))

    print("=== Timings (s) ===")
    for k, v in timings.items():
        print(f"{k:<25} {v:.4f}")
    print("approx MFC weight:", approx_mfc_weight)
    print("coarsened graph weight:", coarsened_weight)


if __name__ == "__main__":
    main()


=== Timings (s) ===
load_and_preprocess       3.9167
partition                 0.0589
component_msts            1091.1405
compute_coarsened_edges   37.9934
mst_coarsened             0.0044
approx MFC weight: 144856.329700768
coarsened graph weight: 144723.81247603893


In [31]:
!zip '/content/outputs_trivago.zip' '/content/outputs_trivago

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [30]:
# zip /content/outputs_trivago
!tar -zcvf output_trivago.tar.gz> outputs_trivago


/bin/bash: line 1: outputs_trivago: Is a directory
